In [1]:
# environment: sceneGraphs_Gaia

# TODO: how to compute the distance between objects: centroid, distance among single points, ...

In [2]:
import numpy as np
import os
import open3d as o3d

import scannet200_constants

In [3]:
pcd_mask3D = o3d.io.read_point_cloud(os.path.join("/local/home/gmarsich/data2TB/LabelMaker/processed_ARKitScenes/40753679/intermediate/scannet200_mask3d_1/mesh_labelled.ply")) # TODO TOSET: change the name of the point cloud to open
o3d.visualization.draw_geometries([pcd_mask3D])

In [4]:
base_path = "/local/home/gmarsich/data2TB/LabelMaker/processed_ARKitScenes/40753679/intermediate/scannet200_mask3d_1" # TODO TOSET
path_predictions = os.path.join(base_path, "predictions.txt") # TODO TOSET: change if necessary

In [5]:
path_pred_masks = "/local/home/gmarsich/data2TB/LabelMaker/processed_ARKitScenes/40753679/intermediate/scannet200_mask3d_1/pred_mask" #TODO TOSET
all_files = os.listdir(path_pred_masks)
txt_files = [f for f in all_files if f.endswith('.txt')]

def extract_number(filename):
    return int(os.path.splitext(filename)[0])

txt_files.sort(key=extract_number)

sorted_txt_paths = [os.path.join(path_pred_masks, f) for f in txt_files]

In [6]:
#
# Get the list of objects that appear in the segmentation (use the files provided from the work of Mask3D)
#

def build_legend(path_predictions):
    with open(path_predictions, 'r') as file:
        lines = file.readlines()

    predictions = []

    for line in lines:
        parts = line.strip().split()
        filename = parts[0]
        file_number = filename.split('/')[1].split('.')[0]
        object_ID = int(parts[1])
        confidence = float(parts[2])

        predictions.append([file_number, object_ID, confidence])

    # Build a list with the info that I need
    objects = []

    for prediction in predictions:
        object_ID = prediction[1]
        
        # Find the object_ID in the objects list
        found = False
        for obj in objects:
            if obj[0] == object_ID:
                obj[1] += 1
                found = True
                break
        
        # If the object_ID was not found, add it to the list with a count of 1
        if not found:
            objects.append([object_ID, 1])


    #
    # Build a big table with the correspondences between VALID_CLASS_IDS_200, CLASS_LABELS_200 and SCANNET_COLOR_MAP_200 from scannet200_constants
    #

    table_scannet200 = []

    for class_id, label in zip(scannet200_constants.VALID_CLASS_IDS_200, scannet200_constants.CLASS_LABELS_200):
        color = scannet200_constants.SCANNET_COLOR_MAP_200[class_id]
        table_scannet200.append((class_id, label, color))

    # An alternative could be to get the colours from the point cloud and search for their assciated IDs (and name of the object) on
        # https://github.com/ScanNet/ScanNet/blob/master/BenchmarkScripts/ScanNet200/scannet200_constants.py


    #
    # Use the big table to add information to the list objects
    #

    # Add label and colour
    for obj in objects:
        object_ID = obj[0]
        
        for entry in table_scannet200:
            class_id, label, color = entry
            if object_ID == class_id:
                obj.append(label)
                obj.append(color)
                break

    # Sort the objects list by the ID (first element of each sublist)
    objects.sort(key=lambda x: x[0])
    return objects

objects = build_legend(path_predictions)

In [7]:
with open(path_predictions, 'r') as file:
    lines = file.readlines()

list_labels = [] # will contain a list of [label, position, color]

def get_centroid(path_pred_mask, pcd_mask3D):
    points = np.asarray(pcd_mask3D.points)
    with open(path_pred_mask, 'r') as f:
        mask = np.array([int(line.strip()) for line in f])
    assert len(mask) == len(points)
    selected_points = points[mask == 1]
    centroid = np.mean(selected_points, axis=0)
    return centroid


for i, path_pred_mask in enumerate(sorted_txt_paths):
    parts = lines[i].strip().split()
    object_ID = int(parts[1])
    label = None
    color = None
    position = get_centroid(path_pred_mask, pcd_mask3D)
    for index in range(len(objects)):
        if objects[index][0] == object_ID:
            label = objects[index][2]
            color = objects[index][3]
            #break
    list_labels.append([position, label, object_ID, color])

In [8]:
print(list_labels)

[[array([ 2.45928517,  0.37223039, -0.09113276]), 'tv', 33, (143.0, 45.0, 115.0)], [array([ 2.69749093, -0.35075306, -0.98032059]), 'chair', 2, (188.0, 189.0, 34.0)], [array([ 2.04590066,  0.04587027, -0.90414485]), 'table', 4, (255.0, 152.0, 150.0)], [array([ 1.5266144 ,  0.63223338, -0.93117788]), 'chair', 2, (188.0, 189.0, 34.0)], [array([ 1.96275443, -2.37670827, -0.52156929]), 'door', 5, (214.0, 39.0, 40.0)], [array([ 1.50364836, -0.43996967,  0.83827772]), 'ceiling', 41, (186.0, 197.0, 62.0)], [array([ 2.85573722, -1.71718747, -0.49505997]), 'window', 16, (197.0, 176.0, 213.0)], [array([ 1.48379265, -0.42478752, -1.02860911]), 'chair', 2, (188.0, 189.0, 34.0)], [array([ 0.22417693,  0.90885343, -0.02731149]), 'whiteboard', 52, (237.0, 80.0, 38.0)], [array([ 0.12986787, -0.99456602, -0.16224432]), 'whiteboard', 52, (237.0, 80.0, 38.0)], [array([ 1.48340331, -0.33068784, -1.00554118]), 'backpack', 48, (143.0, 245.0, 111.0)], [array([ 2.92157881e+00, -1.64357873e+00, -1.94489008e-03

In [9]:
matrix_distances = np.full((len(list_labels), len(list_labels)), np.inf)

for i in range(len(matrix_distances)):
    for j in range(i + 1, len(matrix_distances[0])):
        matrix_distances[i][j] = np.linalg.norm(list_labels[i][0] - list_labels[j][0])


In [10]:
print(matrix_distances)

[[       inf 1.17051358 0.96870344 1.28185496 2.82639031 1.56099792
  2.1647167  1.57025001 2.29951001 2.70173542 1.5108217  2.07006191
  1.75946215 2.67857282 1.75946215 1.75946215]
 [       inf        inf 0.76660465 1.52958465 2.20335767 2.17727342
  1.45865101 1.21691265 2.93464087 2.77063749 1.21451532 1.63671397
  0.79344588 2.43762429 0.79344588 0.79344588]
 [       inf        inf        inf 0.78371596 2.45400981 1.88841585
  1.98281574 0.74362323 2.19823885 2.30306442 0.68445635 2.10594751
  1.55782621 3.04002257 1.55782621 1.55782621]
 [       inf        inf        inf        inf 3.06785384 2.06908674
  2.73432709 1.06236516 1.60929688 2.27785776 0.96675456 2.82643246
  2.3220005  3.75137992 2.3220005  2.3220005 ]
 [       inf        inf        inf        inf        inf 2.41058472
  1.11044527 2.07279724 3.74991443 2.32355434 2.1564336  1.31409004
  2.12615075 2.59303351 2.12615075 2.12615075]
 [       inf        inf        inf        inf        inf        inf
  2.28849711 1.86

In [14]:
threshold = 2 # distance threshold in meters

# Function to create the graph with colors
def create_graph_with_colors(list_labels, matrix_distances):
    vertices = [label[0] for label in list_labels]
    colors = [np.array(label[3]) / 255.0 for label in list_labels]
    
    # Create a point cloud for the vertices
    points = o3d.geometry.PointCloud()
    points.points = o3d.utility.Vector3dVector(vertices)
    points.colors = o3d.utility.Vector3dVector(colors)

    # Create lines for the edges
    edges = []
    for i in range(len(matrix_distances)):
        for j in range(i + 1, len(matrix_distances[0])):
            if matrix_distances[i][j] <= threshold:
                edges.append([i, j])
    
    lines = o3d.geometry.LineSet()
    lines.points = o3d.utility.Vector3dVector(vertices)
    lines.lines = o3d.utility.Vector2iVector(edges)
    lines.paint_uniform_color([0, 0, 0])

    return points, lines


# Function to visualize with custom point size
def visualize_with_custom_point_size(points, lines, point_size=5):
    # Create a visualization window and set point size
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(points)
    vis.add_geometry(lines)

    # Get the render options and set point size
    render_option = vis.get_render_option()
    render_option.point_size = point_size  # Adjust the point size as needed

    # Run the visualization
    vis.run()
    vis.destroy_window()

# Create the graph
points, lines = create_graph_with_colors(list_labels, matrix_distances)

# Visualize the graph with custom point size
visualize_with_custom_point_size(points, lines, point_size=10)
